In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/MyDrive/FYP/CAA_NRC'
%cd /content/gdrive/MyDrive/FYP/CAA_NRC

/content/gdrive/MyDrive/FYP/CAA_NRC


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
url = "CAA_NRC.csv"
df = pd.read_csv(url)
print(df.shape)
print(df)

(3101, 2)
                                                   Text  label
0     muslim immigrant says that we will conquer ger...      0
1     my message to indian muslims kashmir is the sp...      1
2     normally never i abuse anyone on twitter but t...      2
3     heard someone suggesting holding anti caa nrc ...      1
4     The apex court has issued a notice to the Unio...      2
...                                                 ...    ...
3096  save hindus infiltrators anti national go to p...      2
3097  #Breaking | @BhimArmyChief Chandrashekhar Azad...      2
3098  Tamil Nadu CM Edappadi K Palaniswamy says Citi...      0
3099  some more photos of protest meetings against c...      2
3100  In December the Aligarh Muslim University saw ...      1

[3101 rows x 2 columns]


In [5]:
!pip install contractions
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
     words = remove_non_ascii(words)
     words = to_lowercase(words)
     words = remove_punctuation(words)
     #replace numbers is haveing some issue, feel free to explore
    # words = replace_numbers(words)
     #words = remove_stopwords(words)
     return words

def preprocess(sample):
    sample = remove_URL(sample)
    sample = replace_contractions(sample)
    # Tokenize
    words = nltk.word_tokenize(sample)
    
    # Normalize
    return normalize(words)

     |████████████████████████████████| 106 kB 5.4 MB/s 
     |████████████████████████████████| 287 kB 43.0 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
text=df.Text
preprocessed=[]
for i in range(len(text)):
  temp=preprocess(text[i])
  temp=" ".join(temp)
  preprocessed.append(temp)

In [7]:
cols = ['label']
df[cols] = df[cols].apply(LabelEncoder().fit_transform)

X = df.iloc[:, 0].values
y = df.iloc[:,1].values

In [8]:
df["new_text"]=preprocessed

In [9]:
df=df.drop(columns=["Text"])
df

,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...


In [10]:
df.label.value_counts()

1    1394
2    1082
0     625
Name: label, dtype: int64

In [ ]:
df.to_csv("CAA_NRC-preprocessed.csv", index=False)

In [ ]:
df


,label,new_text
0,0,muslim immigrant says that we will conquer ger...
1,1,my message to indian muslims kashmir is the sp...
2,2,normally never i abuse anyone on twitter but t...
3,1,heard someone suggesting holding anti caa nrc ...
4,2,the apex court has issued a notice to the unio...
...,...,...
3096,2,save hindus infiltrators anti national go to p...
3097,2,breaking bhimarmychief chandrashekhar azad has...
3098,0,tamil nadu cm edappadi k palaniswamy says citi...
3099,2,some more photos of protest meetings against c...
